In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='4ed05622-42da-4f06-89d7-f4d8a957bf23', project_access_token='p-791ad9758cdef0730517ae009b6610cf67afb19e')
pc = project.project_context


# MARATÓN BEHIND THE CODE 2020

# DESAFÍO 8 - Digital House

El reto de Digital House tiene que ver con la misión y visión de la empresa; la cual busca transformar la vida de las personas, desarrollando competencias digitales, para que generen un impacto positivo en la sociedad. En ese sentido, Digital House busca a través de este desafío medir la empleabilidad de los cursos que disponibiliza en su plataforma, es decir que tan factible es que un alumno o egresado de Digital House acceda al mercado laboral o consiga un trabajo en su área luego de finalizar uno o varios de los cursos ofrecidos en la plataforma.

Entender que características o variables hacen que una persona sea más o menos empleable es fundamental para la empresa y para generar ese impacto positivo que busca aportar a la sociedad

<hr>

## Instalación de algunas bibliotecas de Python

In [ ]:
!pip install scikit-learn --upgrade
!pip install scipy --upgrade

<hr>

## Descargue el conjunto de datos del desafío en formato .csv

In [ ]:
import pandas as pd

!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset.tail()

## En el conjunto de datos proporcionado, tenemos las siguientes columnas:

* Unnamed: 0
* EDAD
* GENERO
* RESIDENCIA
* NV_ESTUDIO
* ESTUDIO_PREV
* TRACK_DH
* AVG_DH
* MINUTES_DH
* EXPERIENCIA
* **DIAS_EMP = El valor "target" que se va a predecir**

Podemos verificar fácilmente que faltan valores usando el siguiente código:

```df_training_dataset.info()```

Debe manejar cuidadosamente estos valores faltantes antes de crear un modelo de regresión.

In [ ]:
df_training_dataset.info()

<hr>

## Algunos consejos antes de entrenar a un modelo:

1. Manejar valores perdidos

2. Manejar variables categóricas

<hr>

## A continuación, le brindamos una plantilla simple para que sepa cómo estructurar las entradas y salidas de su modelo

### Removiendo columnas

In [ ]:
df_training_dataset_1 = df_training_dataset.drop(columns=['Unnamed: 0'], inplace=False)
df_training_dataset_1.tail()

### Removendo valores NaN

In [ ]:
#df_training_dataset_2 = df_training_dataset_1.dropna()
df_training_dataset_2 = df_training_dataset_1
df_training_dataset_2.tail()

### Codificando variables categóricas

In [ ]:
# Columnas One-hot-encoding del dataset usando el metodo de Pandas ``get_dummies`` (demontración)
df_training_dataset_3 = pd.get_dummies(df_training_dataset_2, columns=['GENERO','TRACK_DH','ESTUDIO_PREV','NV_ESTUDIO','RESIDENCIA'])
df_training_dataset_3.tail()

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
#impute_zeros = SimpleImputer(missing_values=np.nan, strategy='mean',copy=True)
impute_zeros = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0, verbose=0, copy=True)
from sklearn.impute import MissingIndicator
ind = MissingIndicator(missing_values=np.NaN)
indicator = ind.fit_transform(df_training_dataset_3[['EDAD', 'AVG_DH', 'MINUTES_DH', 'EXPERIENCIA']])
indicator = pd.DataFrame(indicator, columns=['m1','m2','m3','m4'])
df_training_dataset_3 = pd.concat([df_training_dataset_3, indicator], axis=1)

impute_zeros.fit(X=df_training_dataset_3)
# Reconstruyendo un Pandas DataFrame con los resultados
df_training_dataset_imputed = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_training_dataset_3
    ),
    columns=df_training_dataset_3.columns
)
df_training_dataset_imputed.info()

### Seleccionando las columnas "features" y "target"

In [ ]:
df_training_dataset_imputed.columns

In [ ]:
features =df_training_dataset_imputed[
    [
        'EDAD', 'AVG_DH', 'MINUTES_DH', 'EXPERIENCIA', 
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 'TRACK_DH_DATA',
       'TRACK_DH_EJECUTIVO', 'TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 'NV_ESTUDIO_POST_GRADUATE',
       'NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'RESIDENCIA_ARGENTINA', 'RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 'm1',
       'm2', 'm3', 'm4'
    ]
]

target = df_training_dataset_imputed['DIAS_EMP']  ## NO CAMBIE EL NOMBRE DE LA VARIABLE "target".

### Dividiendo el conjunto de datos para una "prueba ciega"

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=133)

In [ ]:
X_train = features
y_train = features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_train = poly.fit_transform(X_train)
X_test=poly.transform(X_test)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Entrenando un modelo de regresión lineal simple

In [ ]:
import xgboost as xgb

para = {'colsample_bytree': .5,
        'learning_rate':  0.0297,
        'max_depth': 5,
        'min_child_weight': 5,
        'n_estimators': 286,
        'nthread': -1,
        'silent': 1,
        'subsample': 0.9}
regression_model = xgb.XGBRegressor(**para).fit(X_train, y_train)

### Evaluando el modelo de regresión con el método  "score()" de sklearn

In [ ]:
regression_model.score(X_test, y_test)

<hr>

## Completar los datos necesarios para entregar la solución

### Como entrega de su solución, esperamos los resultados numéricos predichos por su modelo. Como entrada utilizará el archivo "to_be_scored.csv"

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/to_be_scored_digitalhouse.csv
df_to_be_scored = pd.read_csv(r'to_be_scored_digitalhouse.csv')
df_to_be_scored.tail()

# ¡Atención!

### El marco de datos ``to_be_scored`` es su "hoja de evaluación". Tenga en cuenta que la columna "target" no existe en esta muestra, por lo que no se puede utilizar para modelos de entrenamiento basados en el aprendizaje supervisado.

# ¡Atención!

### Debes realizar los mismos pasos de procesamiento previo que hiciste en el conjunto de datos de entrenamiento antes de calificar la "hoja de respuestas"

In [ ]:
df_to_be_scored_1 = df_to_be_scored.drop(columns=['Unnamed: 0'], inplace=False)

df_to_be_scored_2 = df_to_be_scored_1

df_to_be_scored_3 = pd.get_dummies(df_to_be_scored_2, ccolumns=['GENERO','TRACK_DH','ESTUDIO_PREV','NV_ESTUDIO','RESIDENCIA'])
indicator = ind.transform(df_training_dataset_3[['EDAD', 'AVG_DH', 'MINUTES_DH', 'EXPERIENCIA']])
indicator = pd.DataFrame(indicator, columns=['m1','m2','m3','m4'])
df_training_dataset_3 = pd.concat([df_training_dataset_3, indicator], axis=1)

# Reconstruyendo un Pandas DataFrame con los resultados
df_training_dataset_imputed = pd.DataFrame.from_records(
    data=impute_zeros.transform(X=df_training_dataset_3),
    columns=df_training_dataset_3.columns
)
df_training_dataset_imputed.tail()

In [ ]:
df_training_dataset_imputed.columns

<hr>

### Hacer las predicciones con el método "predict()" de sklearn y agregar los resultados en el marco de datos de la "hoja de evaluación"

In [ ]:
X = df_training_dataset_imputed[['EDAD', 'AVG_DH', 'MINUTES_DH', 'EXPERIENCIA',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 'TRACK_DH_DATA',
       'TRACK_DH_EJECUTIVO', 'TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 'NV_ESTUDIO_POST_GRADUATE',
       'NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'RESIDENCIA_ARGENTINA', 'RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO','m1',
       'm2', 'm3', 'm4']]
X = poly.transform(X)
X = scaler.transform(X)
y_pred = regression_model.predict(X)
df_to_be_scored_3['target'] = y_pred
df_to_be_scored_3.tail()

In [ ]:
df_to_be_scored_3.info()

# ¡Atención!

### La columna agregada con los resultados debe llamarse "target", de lo contrario, su envío fallará.

<hr>

### Exportar el marco de datos de resultados como un archivo .csv a su proyecto de Watson Studio.

In [ ]:
project.save_data(file_name="results.csv", data=df_to_be_scored_3.to_csv(index=False))